## 과제 사용법 실습하기
- 논문 사이트 파싱하기
    - https://www.riss.kr/search/Search.do
    - 논문제목, 링크, 학술지명, 발행연도
        - 주제어로 논문 검색 후, 파싱 (빅데이터)
    - 주의

In [2]:
# 1 Page : iStartCount: 0
# 2 Page : iStartCount: 10
# 3 Page : iStartCount: 20
# ..
_params = {
    'arch': 'N',
    'searchGubun': True,
    'viewYn': 'OP',
    'query': '빅데이터',
    'iStartCount': 0,
    'iGroupView': 5,
    'icate': 'all',
    'colName': 're_a_kor',    
    'order': '/DESC',
    'onHanja': False,
    'strSort': 'RANK',
    'pageScale': 10,
    'fsearchMethod': 'search',
    'isFDetailSearch': 'N',
    'sflag': 1,
    'searchQuery': '빅데이터',
    'resultKeyword': '빅데이터',
    'pageNumber': 1}

In [4]:
import requests
import pandas as pd
#from urllib.parse import quote

def riss_parse():
    query = str(input('조회 할 검색어를 입력해주세요 =>'))
    page = str(input('조회 할 페이지 수를 입력해주세요 =>'))

    lst = []

    # =======================================
    # 조회 검색어 설정
    _params['query'] = query
    _params['searchQuery'] = query
    _params['resultKeyword'] = query
    # =======================================
    
    for i in range(1, int(page) + 1):
        url = 'https://www.riss.kr/search/Search.do'
        header = {
            'User-Agent' : 'Mozilla/5.0',
            'Referer' : url
        }

        # print(url)
        print('i : ', i, 'iStartCount : ', str((i - 1) * 10))
        
        # =======================================
        # 조회할 페이지수 관련 - iStartCount 설정
        # - 1Page : iStartCount = 0
        # - 2Page : iStartCount = 10
        # - 3Page : iStartCount = 20
        _params['iStartCount'] = str((i - 1) * 10)
        # =======================================
        
        response = requests.get(url, params = _params, headers=header)
        result = response.text

        from bs4 import BeautifulSoup

        soup = BeautifulSoup(result, 'html.parser')
        #print(soup.prettify())

        items = soup.select("div.srchResultListW > ul > li")
        # print(type(items))
        # print(len(items),'\r\n', items)

        for news in items:
            # 논문제목, 링크, 학술지명, 발행연도
            논문제목 = news.select_one('p.title > a').text.strip()
            링크 = 'https://www.riss.kr' + news.select_one('p.title > a').attrs['href'].strip()    
            학술지명 = news.select_one('p.etc > span:nth-child(4) > a').text.strip()    
            발행연도 = news.select_one('p.etc > span:nth-child(3)').text.strip()        
            
            lst.append([논문제목, 링크, 학술지명, 발행연도])

        # 마지막 페이지 체크
        #if soup.select_one('a.next.\\_nclicks\\:kin\\.next') == None: break

        print(len(lst), lst)

    df = pd.DataFrame(lst, columns = ['논문제목', '링크', '학술지명', '발행연도'])
    df

    df.to_excel('riss_parse.xlsx', index=False)

In [5]:
riss_parse()

i :  1 iStartCount :  0
10 [['지도서비스를 이용한 위치 기반 관광 빅데이터의 시각화', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f77ebda6182557e3b7998d826d417196&keyword=빅데이터', '한국빅데이터서비스학회 논문지', '2014'], ['수산과학 빅데이터 플랫폼 구축과 메타 데이터 관리방안', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=17eb3a02408da46447de9c1710b0298d&keyword=빅데이터', '한국빅데이터학회 학회지', '2019'], ['스마트 물관리를 위한 빅데이터 거버넌스 모델', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d9b0bc216fb83b6c7f7a54760bb41745&keyword=빅데이터', '한국빅데이터학회 학회지', '2018'], ['데이터베이스 연동을 통한 빅데이터 분석결과 가시화', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d2d50f089c50e67fb7998d826d417196&keyword=빅데이터', '한국빅데이터서비스학회 논문지', '2015'], ['효과적인 사이버공간 작전수행을 위한 빅데이터 거버넌스 모델', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=212b486ef21894987f7a54760bb41745&keyword=빅데이터', '한국빅데이터학회 학회지', '201

### riss 디테일 링크 화면 가져오기
- 주의
    - 헤더 정보가 필요할 수 있음
    - 여러 번 수행할 경우 SSL Error 가 발생할 수 있음
    - SSL 인증을 하지 않겠다는 옵션설정
        - response = reuqests.get(link. headers=header, verify=False)
        - Warning 이 출력을 안하고 싶을때
            - import urllib

In [24]:
import urllib3
from urllib3.exceptions import InsecureRequestWarning

urllib3.disable_warnings(InsecureRequestWarning)

In [44]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

query = str(input('조회 할 검색어를 입력해주세요 =>'))
#page = str(input('조회 할 페이지 수를 입력해주세요 =>'))

lst = []
i = 1
# =======================================
# 조회 검색어 설정
_params['query'] = query
_params['searchQuery'] = query
_params['resultKeyword'] = query
# =======================================

url = 'https://www.riss.kr/search/Search.do'
header = {
    'User-Agent' : 'Mozilla/5.0',
    'Referer' : url
}

# print(url)
print('i : ', i, 'iStartCount : ', str((i - 1) * 10))

# =======================================
# 조회할 페이지수 관련 - iStartCount 설정
# - 1Page : iStartCount = 0
# - 2Page : iStartCount = 10
# - 3Page : iStartCount = 20
_params['iStartCount'] = str((i - 1) * 10)
# =======================================

response1 = requests.get(url, params = _params, headers=header)
result1 = response1.text

soup1 = BeautifulSoup(result1, 'html.parser')
#print(soup.prettify())

items = soup1.select("div.srchResultListW > ul > li")
# print(type(items))
# print(len(items),'\r\n', items)

for news in items:
    # 논문제목, 링크, 학술지명, 발행연도
    논문제목 = news.select_one('p.title > a').text.strip()
    링크 = 'https://www.riss.kr' + news.select_one('p.title > a').attrs['href'].strip()
    학술지명 = news.select_one('p.etc > span:nth-child(4) > a').text.strip()
    발행연도 = news.select_one('p.etc > span:nth-child(3)').text.strip()
    
    
    # ====================================================
    # Detail 화면 크롤링
    저자 = ''; 주제어 = ''
    response2 = requests.get(링크, headers=header, verify=True)
    result2 = response2.text
    soup2 = BeautifulSoup(result2)
        
    # Case. Teacher
    # result2 = soup2.find('span', string = '저자').find_next_sibling().text.strip().split(';')
    # namelist = []
    # # for name in result2:
    # #     namelist.append(name.strip())
    # namelist = [name.strip() for name in result2]
    # print(namelist)    
    
    # ㄴ 저자
    # div.infoDetailL > ul > li:nth-child(1) > div > p > a
    result2 = soup2.select('div.infoDetailL > ul > li:nth-child(1) > div > p > a')
    if soup2.find('span', string='저자'):
        저자 = [name.text.strip() for name in result2]
        저자 = list(filter(None, 저자))             # 공백 리스트 제거
        print(저자)
        print(','.join(저자))
        
    # ㄴ 주제어
    # div.infoDetailL > ul > li:nth-child(7) > div > p > a
    result3 = soup2.select('div.infoDetailL > ul > li:nth-child(7) > div > p > a')
    if soup2.find('span', string='주제어'):
        주제어 = [name.text.strip() for name in result3]
        주제어 = list(filter(None, 주제어))         # 공백 리스트 제거
        print(주제어)
        print(','.join(주제어))
    # ====================================================
    
    
    lst.append([논문제목, 링크, 학술지명, 발행연도, ','.join(저자), ','.join(주제어)])

# 마지막 페이지 체크
#if soup.select_one('a.next.\\_nclicks\\:kin\\.next') == None: break

print(len(lst), lst)

df = pd.DataFrame(lst, columns = ['논문제목', '링크', '학술지명', '발행연도', '저자', '주제어'])
df

i :  1 iStartCount :  0
['김경식', '김진국', '조완섭', '류관희', 'Kyeongsik Kim', 'Jinkook Kim', 'Wan-Sup Cho', 'Kwan-Hee Yoo']
김경식,김진국,조완섭,류관희,Kyeongsik Kim,Jinkook Kim,Wan-Sup Cho,Kwan-Hee Yoo
['빅데이터', '전략적 관광 산업', '관광 명소', '빅데이터 시각화', 'Big Data', 'Strategic Tour Industry', 'Chungbuk Tour Place', 'Big Data Visualization']
빅데이터,전략적 관광 산업,관광 명소,빅데이터 시각화,Big Data,Strategic Tour Industry,Chungbuk Tour Place,Big Data Visualization
['김재성', '최영진', '황재동', '조완섭']
김재성,최영진,황재동,조완섭
['수산과학 빅데이터', '빅데이터 플랫폼', '메타 데이터', '표준화', 'Fishery Science Big Data', 'Big Data Platform', 'Meta Data', 'Standardization']
수산과학 빅데이터,빅데이터 플랫폼,메타 데이터,표준화,Fishery Science Big Data,Big Data Platform,Meta Data,Standardization
['최영환', '조완섭', '이경희']
최영환,조완섭,이경희
['데이터 통합', '데이터 거버넌스', '스마트 물관리', '데이터 관리체계', 'Data Integration', 'Data Governance', 'Smart Water Managemen', 'Data Management System']
데이터 통합,데이터 거버넌스,스마트 물관리,데이터 관리체계,Data Integration,Data Governance,Smart Water Managemen,Data Management System
['최성수(Sung Soo Choi)', '지수영(Su 

,논문제목,링크,학술지명,발행연도,저자,주제어
0,지도서비스를 이용한 위치 기반 관광 빅데이터의 시각화,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터서비스학회 논문지,2014,"김경식,김진국,조완섭,류관희,Kyeongsik Kim,Jinkook Kim,Wan-...","빅데이터,전략적 관광 산업,관광 명소,빅데이터 시각화,Big Data,Strateg..."
1,수산과학 빅데이터 플랫폼 구축과 메타 데이터 관리방안,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터학회 학회지,2019,"김재성,최영진,황재동,조완섭","수산과학 빅데이터,빅데이터 플랫폼,메타 데이터,표준화,Fishery Science ..."
2,스마트 물관리를 위한 빅데이터 거버넌스 모델,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터학회 학회지,2018,"최영환,조완섭,이경희","데이터 통합,데이터 거버넌스,스마트 물관리,데이터 관리체계,Data Integrat..."
3,데이터베이스 연동을 통한 빅데이터 분석결과 가시화,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터서비스학회 논문지,2015,"최성수(Sung Soo Choi),지수영(Su Young chi),류관희(Kwan ...","제조 빅데이터,Big Data,Database,Bigdata Visualizatio..."
4,효과적인 사이버공간 작전수행을 위한 빅데이터 거버넌스 모델,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터학회 학회지,2019,"장원구,이경호","빅데이터,빅데이터 거버넌스,사이버공간 정보,사이버공간 작전,사이버공간,Big dat..."
5,분산된 빅데이터베이스 시스템의 동시성 제어 문제의 효율적 관리에 관한 연구,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터서비스학회 논문지,2015,이우기(Wookey Lee),"분산빅데이터베이스,빅데이터 결함관리,빅데이터베이스 동시성제어,Distrubuted ..."
6,의료기관 빅데이터 품질관리의 필요성과 사례 분석,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터학회 학회지,2017,"최혜린,이승원,김영아,이종호,고홍,김현창,Choi, Hye Rin,Lee, Seun...","의료,빅데이터,품질관리"
7,그래프 구조를 갖는 서지 빅데이터의 효율적인 온라인 탐색 및 분석을 지원하는 그래픽...,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터학회 학회지,2020,"유영석,박범준,조선화,이수안,김진호","Graph Data,Bibliographic Big Data,Graphic Inte..."
8,모빌리티 빅데이터 가상결합 분석방법론 연구,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터학회 학회지,2022,"조범철,권기훈,안덕배","Bigdata,Data fusion,Telecommunication data,Mob..."
9,A 은행 사례 분석을 통한 빅데이터 기반 자금세탁방지 시스템 설계,https://www.riss.kr/search/detail/DetailView.d...,한국빅데이터학회 학회지,2016,"김상완,함유근,Kim, Sang-Wan,Hahm, Yu-Kun","빅데이터,자금세탁방지 시스템"


In [45]:
df.to_excel('riss_parse.xlsx', index=False)

## RISS_EXTENSION

In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://www.riss.kr/search/Search.do'
header = {
    'User-Agent' : 'Mozilla/5.0',
    'Referer' : url
}

# =======================================
# 조회 검색어 설정
_params['query'] = '빅데이터'
_params['searchQuery'] = '빅데이터'
_params['resultKeyword'] = '빅데이터'
# =======================================
    
_params['iStartCount'] = '12970'

response = requests.get(url, params = _params, headers=header)
result = response.text

soup = BeautifulSoup(result, 'html.parser')
# print(soup.prettify())

i = 1298
# =========================================
# 마지막 페이지있는 화면 체크!
next1 = soup.select_one(".next1")
if next1 == None:               # 마지막 페이지
    print('next1 = None')
    pages = soup.select("div.paging > a")
    print('len(pages) : ', len(pages))
    print('pages[len(pages) - 1] : ', pages[len(pages) - 2].text)
    
    if i == pages[len(pages) - 2].text:
        break
else:
    print(next1.text)    
# =========================================

#divContent > div > div.rightContent.wd756 > div > div.paging > a


items = soup.select("div.srchResultListW > ul > li")

# next1 버튼
#divContent > div > div.rightContent.wd756 > div > div.paging > a.next1

lst = []
for news in items:
    # 논문제목, 링크, 학술지명, 발행연도
    논문제목 = news.select_one('p.title > a').text.strip()
    링크 = 'https://www.riss.kr' + news.select_one('p.title > a').attrs['href'].strip()    
    학술지명 = news.select_one('p.etc > span:nth-child(4) > a').text.strip()    
    발행연도 = news.select_one('p.etc > span:nth-child(3)').text.strip()        
    
    lst.append([논문제목, 링크, 학술지명, 발행연도])

print(len(lst), lst)
        

next1 = None
len(pages) :  11
pages[len(pages) - 1] :  1298
8 [['K-IFRS 도입이 영업권 손상차손 인식에 미치는 영향', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f1fecb66755d6ad7b36097776a77e665&keyword=빅데이터', '회계학연구', '2015'], ['몽골기업의 이익조정과 원가의 하방경직성', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6f7064df2f00ed937ecd42904f0c5d65&keyword=빅데이터', '산업융합연구', '2022'], ['소비자의 인터넷 접근성 및 활용능력이 상품구매 성향에 미치는 영향', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=197db4d71a709a38ffe0bdc3ef48d419&keyword=빅데이터', '한국언론학보', '2014'], ['한국 글로벌 바이오기업의 경쟁력: 제약바이오산업을 중심으로', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b9413fb1db17c2dce9810257f7042666&keyword=빅데이터', '商品學硏究', '2023'], ['새로운 미니멀 디자인 개발 가능성', 'https://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e6273b56e13b5b507ecd42904f0c5d65&keyword=빅데이터', 'Journal of Integrate

In [25]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#from urllib.parse import quote

# query = str(input('조회 할 검색어를 입력해주세요 =>'))
# page = str(input('조회 할 페이지 수를 입력해주세요 =>'))

query = '머신러닝ai'

lst = []

# =======================================
# 조회 검색어 설정
_params['query'] = query
_params['searchQuery'] = query
_params['resultKeyword'] = query
# =======================================

i = 0
while(True):
    url = 'https://www.riss.kr/search/Search.do'
    header = {
        'User-Agent' : 'Mozilla/5.0',
        'Referer' : url
    }

    i = i + 1
    
    # print(url)
    print('i : ', i, 'iStartCount : ', str((i - 1) * 10))
    
    # =======================================
    # 조회할 페이지수 관련 - iStartCount 설정
    # - 1Page : iStartCount = 0
    # - 2Page : iStartCount = 10
    # - 3Page : iStartCount = 20
    _params['iStartCount'] = str((i - 1) * 10)
    # =======================================
    
    response = requests.get(url, params = _params, headers=header)
    result = response.text

    from bs4 import BeautifulSoup

    soup = BeautifulSoup(result, 'html.parser')
    #print(soup.prettify())

    items = soup.select("div.srchResultListW > ul > li")
    # print(type(items))
    # print(len(items),'\r\n', items)

    for news in items:
        # 논문제목, 링크, 학술지명, 발행연도
        논문제목 = news.select_one('p.title > a').text.strip()
        링크 = 'https://www.riss.kr' + news.select_one('p.title > a').attrs['href'].strip()    
        학술지명 = news.select_one('p.etc > span:nth-child(4) > a').text.strip()    
        발행연도 = news.select_one('p.etc > span:nth-child(3)').text.strip()        
        
        lst.append([논문제목, 링크, 학술지명, 발행연도])

    # 마지막 페이지 체크
    #if soup.select_one('a.next.\\_nclicks\\:kin\\.next') == None: break

    #print(len(lst), lst)
    
    # =========================================
    # 마지막 페이지있는 화면 체크!
    next1 = soup.select_one(".next1")
    if next1 == None:               # 마지막 페이지
        print('next1 = None')
        pages = soup.select("div.paging > a")
        print('i : ', i, ', len(pages) : ', len(pages))
        print('pages[len(pages) - 2] : ', pages[len(pages) - 2].text)
        
        if i == pages[len(pages) - 2].text:
            break
    else:
        print(next1.text)
    # =========================================


df = pd.DataFrame(lst, columns = ['논문제목', '링크', '학술지명', '발행연도'])
df

#df.to_excel('riss_parse.xlsx', index=False)

i :  1 iStartCount :  0
다음 페이지로
i :  2 iStartCount :  10
다음 페이지로
i :  3 iStartCount :  20
다음 페이지로
i :  4 iStartCount :  30
다음 페이지로
i :  5 iStartCount :  40
다음 페이지로
i :  6 iStartCount :  50
다음 페이지로
i :  7 iStartCount :  60
다음 페이지로
i :  8 iStartCount :  70
다음 페이지로
i :  9 iStartCount :  80
다음 페이지로
i :  10 iStartCount :  90
다음 페이지로
i :  11 iStartCount :  100
다음 페이지로
i :  12 iStartCount :  110
다음 페이지로
i :  13 iStartCount :  120
다음 페이지로
i :  14 iStartCount :  130
다음 페이지로
i :  15 iStartCount :  140
다음 페이지로
i :  16 iStartCount :  150
다음 페이지로
i :  17 iStartCount :  160
다음 페이지로
i :  18 iStartCount :  170
다음 페이지로
i :  19 iStartCount :  180
다음 페이지로
i :  20 iStartCount :  190
다음 페이지로
i :  21 iStartCount :  200
다음 페이지로
i :  22 iStartCount :  210
다음 페이지로
i :  23 iStartCount :  220
다음 페이지로
i :  24 iStartCount :  230
다음 페이지로
i :  25 iStartCount :  240
다음 페이지로
i :  26 iStartCount :  250
다음 페이지로
i :  27 iStartCount :  260
다음 페이지로
i :  28 iStartCount :  270
다음 페이지로
i :  29 iStartCount :  280
다음 페이지로
i :  

KeyboardInterrupt: 

In [26]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#from urllib.parse import quote

# query = str(input('조회 할 검색어를 입력해주세요 =>'))
# page = str(input('조회 할 페이지 수를 입력해주세요 =>'))

query = '머신러닝ai'

lst = []

# =======================================
# 조회 검색어 설정
_params['query'] = query
_params['searchQuery'] = query
_params['resultKeyword'] = query
# =======================================

i = 0
while(True):
    url = 'https://www.riss.kr/search/Search.do'
    header = {
        'User-Agent' : 'Mozilla/5.0',
        'Referer' : url
    }

    i = i + 1
    
    # print(url)
    print('i : ', i, 'iStartCount : ', str((i - 1) * 10))
    
    # =======================================
    # 조회할 페이지수 관련 - iStartCount 설정
    # - 1Page : iStartCount = 0
    # - 2Page : iStartCount = 10
    # - 3Page : iStartCount = 20
    _params['iStartCount'] = str((i - 1) * 10)
    # =======================================
    
    response = requests.get(url, params = _params, headers=header)
    result = response.text

    from bs4 import BeautifulSoup

    soup = BeautifulSoup(result, 'html.parser')
    #print(soup.prettify())

    items = soup.select("div.srchResultListW > ul > li")
    # print(type(items))
    # print(len(items),'\r\n', items)
    
    # 마지막 페이지??
    if len(items) == 0: break
    
    for news in items:
        # 논문제목, 링크, 학술지명, 발행연도
        논문제목 = news.select_one('p.title > a').text.strip()
        링크 = 'https://www.riss.kr' + news.select_one('p.title > a').attrs['href'].strip()    
        학술지명 = news.select_one('p.etc > span:nth-child(4) > a').text.strip()    
        발행연도 = news.select_one('p.etc > span:nth-child(3)').text.strip()        
        
        lst.append([논문제목, 링크, 학술지명, 발행연도])

    #print(len(lst), lst)
    
    # =========================================
    # 마지막 페이지있는 화면 체크!
    next1 = soup.select_one(".next1")
    if next1 == None:               # 마지막 페이지
        print('next1 = None')
        pages = soup.select("div.paging > a")
        print('i : ', i, ', len(pages) : ', len(pages))
        print('pages[len(pages) - 2] : ', pages[len(pages) - 2].text)
        
        if i == pages[len(pages) - 2].text:
            break
    else:
        print(next1.text)
    # =========================================


df = pd.DataFrame(lst, columns = ['논문제목', '링크', '학술지명', '발행연도'])
df

#df.to_excel('riss_parse.xlsx', index=False)

i :  1 iStartCount :  0
다음 페이지로
i :  2 iStartCount :  10
다음 페이지로
i :  3 iStartCount :  20
다음 페이지로
i :  4 iStartCount :  30
다음 페이지로
i :  5 iStartCount :  40
다음 페이지로
i :  6 iStartCount :  50
다음 페이지로
i :  7 iStartCount :  60
다음 페이지로
i :  8 iStartCount :  70
다음 페이지로
i :  9 iStartCount :  80
다음 페이지로
i :  10 iStartCount :  90
다음 페이지로
i :  11 iStartCount :  100
다음 페이지로
i :  12 iStartCount :  110
다음 페이지로
i :  13 iStartCount :  120
다음 페이지로
i :  14 iStartCount :  130
다음 페이지로
i :  15 iStartCount :  140
다음 페이지로
i :  16 iStartCount :  150
다음 페이지로
i :  17 iStartCount :  160
다음 페이지로
i :  18 iStartCount :  170
다음 페이지로
i :  19 iStartCount :  180
다음 페이지로
i :  20 iStartCount :  190
다음 페이지로
i :  21 iStartCount :  200
다음 페이지로
i :  22 iStartCount :  210
다음 페이지로
i :  23 iStartCount :  220
다음 페이지로
i :  24 iStartCount :  230
다음 페이지로
i :  25 iStartCount :  240
다음 페이지로
i :  26 iStartCount :  250
다음 페이지로
i :  27 iStartCount :  260
다음 페이지로
i :  28 iStartCount :  270
다음 페이지로
i :  29 iStartCount :  280
다음 페이지로
i :  

,논문제목,링크,학술지명,발행연도
0,AI 머신러닝 알고리즘을 적용한 한국여자프로골프(KLPGA) 선수의 상위 10위 이...,https://www.riss.kr/search/detail/DetailView.d...,한국스포츠산업경영학회지,2023
1,AI 머신러닝을 이용한 자동차 고장진단 시스템 설계 및 개발,https://www.riss.kr/search/detail/DetailView.d...,차세대컨버전스정보서비스기술논문지,2018
2,Open STT API와 머신러닝을 이용한 AI 보이스피싱 예방 솔루션,https://www.riss.kr/search/detail/DetailView.d...,한국정보처리학회 학술대회논문집,2022
3,인공지능 머신러닝 교육이 고등학생의 AI 리터러시에 미치는 영향,https://www.riss.kr/search/detail/DetailView.d...,정보교육학회논문지,2023
4,남한강수계 내 클로로필-a 예측을 위한 머신러닝 및 해석가능한 AI의 적용,https://www.riss.kr/search/detail/DetailView.d...,한국물환경학회·대한상하수도학회 공동 춘계학술발표회,2024
...,...,...,...,...
499,XAI를 활용한 특별관리해역의 연안오염총량관리제 오염물질 대상항목 선정 및 해양오염...,https://www.riss.kr/search/detail/DetailView.d...,한국해양환경·에너지학회지,2023
500,인공지능 규제의 법체계,https://www.riss.kr/search/detail/DetailView.d...,土地公法硏究,2021
501,인공지능 챗봇에 대한 선문답 알고리즘의 데이터 ‒ 심리치료 상담챗봇을 중심으로 ‒,https://www.riss.kr/search/detail/DetailView.d...,韓國佛敎學,2017
502,국내 스포츠 애널리틱스 연구의 현재와 미래 : 경기데이터 활용 분석에 대한 체계적 ...,https://www.riss.kr/search/detail/DetailView.d...,한국스포츠산업경영학회지,2024
